<h1>导入数据</h1>

In [66]:
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from time import strftime, localtime
from scipy import stats
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from joblib import dump, load

<h1>数据处理</h1>

<h2>数据预处理</h2>

<h3>conf_info数据预处理</h3>

In [67]:
cons_info = pd.read_csv("../data/训练组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info

,ID,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE,IS_FLAG
,,,,,,,,,,,,,,
1,329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
2,329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
3,329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
4,329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
5,329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8836,2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,NaN,120.0,2019/12/2 16:12:00,非临时用电,NaN,0
8837,2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,NaN,120.0,2019/12/3 16:33:00,非临时用电,NaN,0
8838,2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,NaN,120.0,2019/12/18 14:42:00,非临时用电,NaN,0


In [68]:
cons_info = cons_info.set_index("ID").drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,120.0,2019/12/2 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,120.0,2019/12/3 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,120.0,2019/12/18 14:42:00,非临时用电,0


In [69]:
cons_info["BUILD_DATE"] = pd.to_datetime(cons_info["BUILD_DATE"])
cons_info["LAST_CHK_DATE"] = pd.to_datetime(cons_info["LAST_CHK_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [70]:
cons_info["SHIFT_NO"] = cons_info["SHIFT_NO"].fillna(0)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [71]:
cons_info["now"] = "2021-12-31"
cons_info["now"] = pd.to_datetime(cons_info["now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now
ID,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31


In [72]:
cons_info["live_days"] = (cons_info["now"] - cons_info["BUILD_DATE"]).map(lambda x: x.days)
cons_info["check_days"] = (cons_info["now"] - cons_info["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now,live_days,check_days
ID,,,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31,743,743


In [73]:
cons_info = cons_info.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,IS_FLAG,live_days,check_days
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,240.0,非临时用电,0,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,120.0,非临时用电,0,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,743,743


In [74]:
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  8840 non-null   object 
 1   VOLT_NAME       8840 non-null   object 
 2   PRC_NAME        8840 non-null   object 
 3   CONTRACT_CAP    8840 non-null   float64
 4   RUN_CAP         8840 non-null   float64
 5   SHIFT_NO        8840 non-null   float64
 6   CHK_CYCLE       8838 non-null   float64
 7   TMP_NAME        8840 non-null   object 
 8   IS_FLAG         8840 non-null   int64  
 9   live_days       8840 non-null   int64  
 10  check_days      8840 non-null   int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 828.8+ KB


In [75]:
cons_info = pd.get_dummies(cons_info)
cons_info

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,...,0,0,0,0,0,0,1,0,1,1
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,...,0,0,0,0,0,1,0,0,1,1
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,...,0,0,0,0,0,0,1,0,1,1


In [76]:
cons_info["CHK_CYCLE"] = cons_info["CHK_CYCLE"].fillna(cons_info["CHK_CYCLE"].median())
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                8840 non-null   float64
 1   RUN_CAP                     8840 non-null   float64
 2   SHIFT_NO                    8840 non-null   float64
 3   CHK_CYCLE                   8840 non-null   float64
 4   IS_FLAG                     8840 non-null   int64  
 5   live_days                   8840 non-null   int64  
 6   check_days                  8840 non-null   int64  
 7   ELEC_TYPE_NAME_乡村居民生活用电     8840 non-null   uint8  
 8   ELEC_TYPE_NAME_商业用电         8840 non-null   uint8  
 9   ELEC_TYPE_NAME_城镇居民生活用电     8840 non-null   uint8  
 10  ELEC_TYPE_NAME_学校教学和学生生活用电  8840 non-null   uint8  
 11  ELEC_TYPE_NAME_居民生活用电       8840 non-null   uint8  
 12  ELEC_TYPE_NAME_普通工业         8840 non-null   uint8  
 13  ELEC_TYPE_NAME_非居民照

<h3>elec_month数据预处理</h3>

In [77]:
cons_info["pq_z_mean"] = 0
cons_info["pq_z_median"] = 0
cons_info["pq_z_std"] = 0
cons_info["pq_z_std/mean"] = 0
cons_info["pq_z_entropy"] = 0
cons_info["pq_fg_distance"] = 0
cons_info["pq_gp_diatance"] = 0
cons_info["pq_fp_distance"] = 0
cons_info["pq_fg_cosine"] = 0
cons_info["pq_gp_cosine"] = 0
cons_info["pq_fp_cosine"] = 0
cons_info["pq_fz_cosine"] = 0
cons_info["pq_gz_cosine"] = 0
cons_info["pq_pz_cosine"] = 0
cons_info["variation_month"] = 0
cons_info["pq_z_mean_4500"] = 0
cons_info["variation_month_0085"] = 0

In [78]:
def entropy(data, bins: int = 10) -> float:
    """
    计算一个序列的熵\n
    """
    nums = np.histogram(data, bins=bins)[0]
    probs = nums / sum(nums)
    return stats.entropy(probs)

In [79]:
def cosine(a, b) -> float:
    """
    计算余弦距离\n
    """
    if np.linalg.norm(a) != 0 and np.linalg.norm(b) != 0:
        return distance.cosine(a, b)
    elif np.linalg.norm(a) == 0 and np.linalg.norm(b) == 0:
        return 0
    else:
        return 1

In [80]:
def variation(df: pd.DataFrame) -> float:
    """
    计算变异系数\n
    """
    tt =  df.std(axis=1) / df.mean(axis=1)
    tt[tt.apply(lambda x: np.isinf(x))] = 0
    return tt.mean(axis=0)

In [81]:
elec_month = pd.read_csv("../data/训练组_比特币挖矿_月用电明细（20211217）.csv")
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [82]:
elec_month["ym"] = elec_month["ym"].astype(str)
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [83]:
for cons_id in tqdm(cons_info.index):
    df = elec_month[elec_month["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill")
    cons_info.loc[cons_id, "pq_z_mean"] = df["pq_z"].mean()
    cons_info.loc[cons_id, "pq_z_median"] = df["pq_z"].median()
    cons_info.loc[cons_id, "pq_z_std"] = df["pq_z"].std()
    cons_info.loc[(cons_info.index == cons_id) & (cons_info["pq_z_mean"] > 0), "pq_z_std/mean"] = cons_info["pq_z_std"] / cons_info["pq_z_mean"]
    cons_info.loc[cons_id, "pq_z_entropy"] = entropy(df["pq_z"])
    cons_info.loc[cons_id, "pq_fg_distance"] = distance.euclidean(df["pq_f"], df["pq_g"])
    cons_info.loc[cons_id, "pq_gp_diatance"] = distance.euclidean(df["pq_g"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fp_distance"] = distance.euclidean(df["pq_f"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fg_cosine"] = cosine(df["pq_f"], df["pq_g"])
    cons_info.loc[cons_id, "pq_gp_cosine"] = cosine(df["pq_g"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fp_cosine"] = cosine(df["pq_f"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fz_cosine"] = cosine(df["pq_f"], df["pq_z"])
    cons_info.loc[cons_id, "pq_gz_cosine"] = cosine(df["pq_g"], df["pq_z"])
    cons_info.loc[cons_id, "pq_pz_cosine"] = cosine(df["pq_p"], df["pq_z"])
    cons_info.loc[cons_id, "variation_month"] = variation(df[["pq_f", "pq_g", "pq_p"]])
    cons_info.loc[cons_id, "pq_z_mean_4500"] = (cons_info.loc[cons_id, "pq_z_mean"] > 4500).astype(float)
    cons_info.loc[cons_id, "variation_month_0085"] = (cons_info.loc[cons_id, "variation_month"] < 0.085).astype(float)
    
cons_info

100%|██████████████████████████████████| 8840/8840 [01:19<00:00, 111.09it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,pq_fp_distance,pq_fg_cosine,pq_gp_cosine,pq_fp_cosine,pq_fz_cosine,pq_gz_cosine,pq_pz_cosine,variation_month,pq_z_mean_4500,variation_month_0085
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,2425.555813,0.014260,1.030399,1.069618,0.001869,0.005819,1.054778,0.985564,0.0,0.0
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,3495.833377,0.009833,1.049541,1.049255,0.001004,0.004561,1.048902,1.036198,0.0,0.0
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,7997.614144,0.000281,0.949959,0.947601,0.711070,0.713365,0.028826,1.653016,0.0,0.0
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,...,3726.984304,0.010028,0.850390,0.860356,0.000896,0.004936,0.856495,1.078517,0.0,0.0
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,5675.094449,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.732051,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,...,5183.746811,0.000915,1.078510,1.073679,0.000104,0.000402,1.074872,0.991196,0.0,0.0
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,...,2021.216218,0.005729,1.143152,1.123854,0.000500,0.002849,1.128561,0.998110,0.0,0.0
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,...,9.848858,0.086188,1.000000,1.000000,0.000711,0.071523,1.000000,1.605958,0.0,0.0


<h3>elec_day数据预处理</h3>

In [84]:
cons_info["kwh_mean"] = 0
cons_info["kwh_median"] = 0
cons_info["kwh_std"] = 0
cons_info["kwh_std/mean"] = 0
cons_info["kwh_entropy"] = 0

cons_info["kwh_pap_r23_distance"] = 0
cons_info["kwh_pap_r24_distance"] = 0
cons_info["kwh_pap_r34_distance"] = 0

cons_info["kwh_pap_r23_cosine"] = 0
cons_info["kwh_pap_r24_cosine"] = 0
cons_info["kwh_pap_r34_cosine"] = 0

cons_info["kwh_pap_r1_cosine"] = 0
cons_info["kwh_pap_r2_cosine"] = 0
cons_info["kwh_pap_r3_cosine"] = 0
cons_info["kwh_pap_r4_cosine"] = 0
cons_info["variation_day"] = 0

In [85]:
elec_day = pd.read_csv("../data/训练组_比特币挖矿_日用电明细（20211217）.csv")
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22 00:00:00,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23 00:00:00,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24 00:00:00,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25 00:00:00,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26 00:00:00,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05 00:00:00,7.22,NaN,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06 00:00:00,7.23,NaN,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07 00:00:00,7.20,NaN,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08 00:00:00,7.23,NaN,0.0,4.50,0.0,2.73


In [86]:
elec_day["rq"] = pd.to_datetime(elec_day["rq"])
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05,7.22,NaN,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06,7.23,NaN,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07,7.20,NaN,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08,7.23,NaN,0.0,4.50,0.0,2.73


In [87]:
for cons_id in tqdm(cons_info.index):
    df = elec_day[elec_day["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("rq").set_index("rq")
    df = df.fillna(method="ffill").fillna(method="bfill")
    cons_info.loc[cons_id, "kwh_mean"] = df["kwh"].mean()
    cons_info.loc[cons_id, "kwh_median"] = df["kwh"].median()
    cons_info.loc[cons_id, "kwh_std"] = df["kwh"].std()
    cons_info.loc[(cons_info.index == cons_id) & (cons_info["kwh_mean"] > 0), "kwh_std/mean"] = cons_info["kwh_std"] / cons_info["kwh_mean"]
    cons_info.loc[cons_id, "kwh_entropy"] = entropy(df["kwh"])

    cons_info.loc[cons_id, "kwh_pap_r23_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info.loc[cons_id, "kwh_pap_r24_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info.loc[cons_id, "kwh_pap_r34_distance"] = distance.euclidean(df["kwh_pap_r3"], df["kwh_pap_r4"])

    cons_info.loc[cons_id, "kwh_pap_r23_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info.loc[cons_id, "kwh_pap_r24_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info.loc[cons_id, "kwh_pap_r34_cosine"] = cosine(df["kwh_pap_r3"], df["kwh_pap_r4"])
    
    cons_info.loc[cons_id, "kwh_pap_r1_cosine"] = cosine(df["kwh_pap_r1"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r2_cosine"] = cosine(df["kwh_pap_r2"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r3_cosine"] = cosine(df["kwh_pap_r3"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r4_cosine"] = cosine(df["kwh_pap_r4"], df["kwh"])
    
    cons_info.loc[cons_id, "variation_day"] = variation(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
cons_info

100%|███████████████████████████████████| 8840/8840 [01:29<00:00, 98.98it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,kwh_pap_r24_distance,kwh_pap_r34_distance,kwh_pap_r23_cosine,kwh_pap_r24_cosine,kwh_pap_r34_cosine,kwh_pap_r1_cosine,kwh_pap_r2_cosine,kwh_pap_r3_cosine,kwh_pap_r4_cosine,variation_day
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,82.996603,78.143830,1.0,0.112179,1.0,1.0,0.013884,1.0,0.048088,1.039291
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,...,143.991545,119.935441,1.0,0.042256,1.0,1.0,0.004351,1.0,0.019658,1.045283
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,348.667828,122.562718,1.0,0.039354,1.0,1.0,0.001687,1.0,0.024876,1.236592
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,...,161.198520,114.334646,1.0,0.034102,1.0,1.0,0.002899,1.0,0.017558,1.103814
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,...,164.456730,106.217331,1.0,0.045007,1.0,1.0,0.003703,1.0,0.023050,1.094278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,...,142.254446,154.273303,1.0,0.046646,1.0,1.0,0.005853,1.0,0.019615,0.962473
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,...,70.953974,60.024030,1.0,0.027986,1.0,1.0,0.002854,1.0,0.013023,0.965715
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,NaN


<h2>数据集划分</h2>

In [88]:
x = cons_info.drop(columns=["IS_FLAG"])
y = cons_info["IS_FLAG"]
print(x.shape, y.shape)

(8840, 53) (8840,)


<h2>标准化</h2>

In [89]:
ss = StandardScaler()
x = ss.fit_transform(x)
x

array([[-0.31260749, -0.31917163,  1.05812375, ...,  0.53828596,
        -0.16223824,  0.16172333],
       [-0.31260749, -0.31917163,  1.05812375, ...,  0.53828596,
        -0.31772669,  0.18027779],
       [-0.2464131 , -0.25086739,  1.05812375, ...,  0.53828596,
        -0.28918791,  0.77265896],
       ...,
       [ 0.2665934 ,  0.27849043, -0.86811471, ..., -1.93452962,
        -0.42523935,         nan],
       [ 0.2665934 ,  0.27849043, -0.86811471, ...,  0.53828596,
         0.31572979,  0.13604547],
       [ 0.11765603,  0.1248059 , -0.86811471, ...,  0.53828596,
        -0.40238261,  0.25643336]])

<h1>做出来一个模型</h1>

In [26]:
model = LGBMClassifier(n_estimators=500, objective="binary", learning_rate=0.1, class_weight="balanced")
model.fit(x, y)

LGBMClassifier(class_weight='balanced', n_estimators=500, objective='binary')

In [27]:
accuracy_score(y, model.predict(x))

1.0

In [28]:
f1_score(y, model.predict(x))

1.0

In [29]:
roc_auc_score(y, model.predict(x))

1.0

In [30]:
dump(model, "../models/lightBGM-20220210-1.pkl")

['../models/lightBGM-20220210-1.pkl']

<h1>处理测试数据</h1>

<h2>数据预处理</h2>

<h3>档案数据预处理</h3>

In [91]:
cons_info_test = pd.read_csv("../data/测试组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE
ID,,,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,NaN,24,1998/12/1,非临时用电,NaN
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,1998/12/1,NaN,24,1998/12/1,非临时用电,NaN
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999/4/1,NaN,24,1999/4/1,非临时用电,NaN
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999/3/1,NaN,36,1999/3/1,非临时用电,NaN
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,NaN,24,1998/12/1,非临时用电,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,2019/11/8 12:42:00,NaN,120,2019/11/8 12:42:00,非临时用电,NaN
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,NaN,2019/11/18 14:08:00,NaN,120,2019/11/18 14:08:00,非临时用电,NaN
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,NaN,2019/11/18 14:08:00,NaN,120,2019/11/18 14:08:00,非临时用电,NaN


In [92]:
cons_info_test = cons_info_test.drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,24,1998/12/1,非临时用电
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,1998/12/1,24,1998/12/1,非临时用电
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999/4/1,24,1999/4/1,非临时用电
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999/3/1,36,1999/3/1,非临时用电
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,24,1998/12/1,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,2019/11/8 12:42:00,120,2019/11/8 12:42:00,非临时用电
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,NaN,2019/11/18 14:08:00,120,2019/11/18 14:08:00,非临时用电
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,NaN,2019/11/18 14:08:00,120,2019/11/18 14:08:00,非临时用电


In [93]:
cons_info_test["BUILD_DATE"] = pd.to_datetime(cons_info_test["BUILD_DATE"])
cons_info_test["LAST_CHK_DATE"] = pd.to_datetime(cons_info_test["LAST_CHK_DATE"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,NaN,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,NaN,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电


In [94]:
cons_info_test["SHIFT_NO"] = cons_info_test["SHIFT_NO"].fillna(0)
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电


In [95]:
cons_info_test["now"] = "2021-12-31"
cons_info_test["now"] = pd.to_datetime(cons_info_test["now"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,now
ID,,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电,2021-12-31
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电,2021-12-31
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电,2021-12-31
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31


In [96]:
cons_info_test["live_days"] = (cons_info_test["now"] - cons_info_test["BUILD_DATE"]).map(lambda x: x.days)
cons_info_test["check_days"] = (cons_info_test["now"] - cons_info_test["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,now,live_days,check_days
ID,,,,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31,8431,8431
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31,8431,8431
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电,2021-12-31,8310,8310
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电,2021-12-31,8341,8341
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31,8431,8431
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电,2021-12-31,783,783
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31,773,773
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31,773,773


In [97]:
cons_info_test = cons_info_test.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,live_days,check_days
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8431,8431
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,24,非临时用电,8431,8431
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8310,8310
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,36,非临时用电,8341,8341
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8431,8431
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,120,非临时用电,783,783
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,120,非临时用电,773,773
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,120,非临时用电,773,773


In [98]:
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15379 entries, 179406029 to 2852368013
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  15379 non-null  object 
 1   VOLT_NAME       15379 non-null  object 
 2   PRC_NAME        15379 non-null  object 
 3   CONTRACT_CAP    15379 non-null  float64
 4   RUN_CAP         15379 non-null  float64
 5   SHIFT_NO        15379 non-null  float64
 6   CHK_CYCLE       15379 non-null  int64  
 7   TMP_NAME        15379 non-null  object 
 8   live_days       15379 non-null  int64  
 9   check_days      15379 non-null  int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 1.3+ MB


In [99]:
cons_info_test = pd.get_dummies(cons_info_test)
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,VOLT_NAME_交流6kV,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民生活1_10kV(合表),PRC_NAME_居民生活<1kV(合表),TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,0,0,0,1,0,0,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1


In [100]:
cons_info_test["CHK_CYCLE"] = cons_info_test["CHK_CYCLE"].fillna(cons_info_test["CHK_CYCLE"].median())
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15379 entries, 179406029 to 2852368013
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                15379 non-null  float64
 1   RUN_CAP                     15379 non-null  float64
 2   SHIFT_NO                    15379 non-null  float64
 3   CHK_CYCLE                   15379 non-null  int64  
 4   live_days                   15379 non-null  int64  
 5   check_days                  15379 non-null  int64  
 6   ELEC_TYPE_NAME_乡村居民生活用电     15379 non-null  uint8  
 7   ELEC_TYPE_NAME_农业生产用电       15379 non-null  uint8  
 8   ELEC_TYPE_NAME_商业用电         15379 non-null  uint8  
 9   ELEC_TYPE_NAME_城镇居民生活用电     15379 non-null  uint8  
 10  ELEC_TYPE_NAME_大工业用电        15379 non-null  uint8  
 11  ELEC_TYPE_NAME_学校教学和学生生活用电  15379 non-null  uint8  
 12  ELEC_TYPE_NAME_居民生活用电       15379 non-null  uint8  
 13  ELEC_TYPE_NAME_普通工

In [101]:
cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] = cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] + cons_info_test["PRC_NAME_居民生活1_10kV(合表)"]
cons_info_test = cons_info_test.drop(columns=["PRC_NAME_居民生活1_10kV(合表)"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,VOLT_NAME_交流6kV,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民生活<1kV(合表),TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1


In [102]:
cons_info_test = cons_info_test.rename(columns={"PRC_NAME_居民生活<1kV(合表)": "PRC_NAME_居民合表电价(不满1千伏）"})
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,VOLT_NAME_交流6kV,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1


In [103]:
cons_info_test["VOLT_NAME_交流10kV"] = cons_info_test["VOLT_NAME_交流10kV"] + cons_info_test["VOLT_NAME_交流6kV"]
cons_info_test = cons_info_test.drop(columns=["VOLT_NAME_交流6kV"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_农业生产用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,...,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,0,0,0,0,0,1,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,0,1,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,0,0,0,0,0,0,1,0,1,1


In [104]:
cons_info_test["ELEC_TYPE_NAME_普通工业"] = cons_info_test["ELEC_TYPE_NAME_普通工业"] + cons_info_test["ELEC_TYPE_NAME_大工业用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_大工业用电"])
cons_info_test["ELEC_TYPE_NAME_非工业"] = cons_info_test["ELEC_TYPE_NAME_非工业"] + cons_info_test["ELEC_TYPE_NAME_农业生产用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_农业生产用电"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406030,12.0,12.0,0.0,24,8431,8431,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406094,2.0,2.0,1.0,24,8310,8310,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406097,4.0,4.0,1.0,36,8341,8341,0,0,1,0,0,0,0,0,0,1,0,0,1,1
179406099,2.0,2.0,1.0,24,8431,8431,0,0,1,0,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,0,0,0,0,0,1,0,0,1,1
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,0,0,0,0,0,0,1,0,1,1
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,0,0,0,0,0,0,1,0,1,1


<h3>月电量处理</h3>

In [105]:
cons_info_test["pq_z_mean"] = 0
cons_info_test["pq_z_median"] = 0
cons_info_test["pq_z_std"] = 0
cons_info_test["pq_z_std/mean"] = 0
cons_info_test["pq_z_entropy"] = 0
cons_info_test["pq_fg_distance"] = 0
cons_info_test["pq_gp_diatance"] = 0
cons_info_test["pq_fp_distance"] = 0
cons_info_test["pq_fg_cosine"] = 0
cons_info_test["pq_gp_cosine"] = 0
cons_info_test["pq_fp_cosine"] = 0
cons_info_test["pq_fz_cosine"] = 0
cons_info_test["pq_gz_cosine"] = 0
cons_info_test["pq_pz_cosine"] = 0
cons_info_test["variation_month"] = 0
cons_info_test["pq_z_mean_4500"] = 0
cons_info_test["variation_month_0085"] = 0

In [106]:
elec_month_test = pd.read_csv("../data/测试组_比特币挖矿_月用电明细（20211217）.csv")
elec_month_test

,id,ym,pq_f,pq_g,pq_p,pq_z
0,179438260,202001,244,111,1,356
1,179438260,202002,217,108,-1,324
2,179438260,202003,280,90,0,370
3,179438260,202004,338,118,1,457
4,179438260,202005,297,120,-1,416
...,...,...,...,...,...,...
338333,2880712108,202106,474,675,0,1149
338334,2880712108,202107,342,881,0,1223
338335,2880712108,202108,293,772,0,1065
338336,2880712108,202109,418,570,-1,987


In [107]:
elec_month_test["ym"] = elec_month_test["ym"].astype(str)
elec_month_test

,id,ym,pq_f,pq_g,pq_p,pq_z
0,179438260,202001,244,111,1,356
1,179438260,202002,217,108,-1,324
2,179438260,202003,280,90,0,370
3,179438260,202004,338,118,1,457
4,179438260,202005,297,120,-1,416
...,...,...,...,...,...,...
338333,2880712108,202106,474,675,0,1149
338334,2880712108,202107,342,881,0,1223
338335,2880712108,202108,293,772,0,1065
338336,2880712108,202109,418,570,-1,987


In [108]:
for cons_id in tqdm(cons_info_test.index):
    df = elec_month_test[elec_month_test["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill")
    cons_info_test.loc[cons_id, "pq_z_mean"] = df["pq_z"].mean()
    cons_info_test.loc[cons_id, "pq_z_median"] = df["pq_z"].median()
    cons_info_test.loc[cons_id, "pq_z_std"] = df["pq_z"].std()
    cons_info_test.loc[(cons_info_test.index == cons_id) & (cons_info_test["pq_z_mean"] > 0), "pq_z_std/mean"] = cons_info_test["pq_z_std"] / cons_info_test["pq_z_mean"]
    cons_info_test.loc[cons_id, "pq_z_entropy"] = entropy(df["pq_z"])
    
    cons_info_test.loc[cons_id, "pq_fg_distance"] = distance.euclidean(df["pq_f"], df["pq_g"])
    cons_info_test.loc[cons_id, "pq_gp_diatance"] = distance.euclidean(df["pq_g"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fp_distance"] = distance.euclidean(df["pq_f"], df["pq_p"])
    
    cons_info_test.loc[cons_id, "pq_fg_cosine"] = cosine(df["pq_f"], df["pq_g"])
    cons_info_test.loc[cons_id, "pq_gp_cosine"] = cosine(df["pq_g"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fp_cosine"] = cosine(df["pq_f"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fz_cosine"] = cosine(df["pq_f"], df["pq_z"])
    cons_info_test.loc[cons_id, "pq_gz_cosine"] = cosine(df["pq_g"], df["pq_z"])
    cons_info_test.loc[cons_id, "pq_pz_cosine"] = cosine(df["pq_p"], df["pq_z"])
    
    cons_info_test.loc[cons_id, "variation_month"] = variation(df[["pq_f", "pq_g", "pq_p"]])
    cons_info_test.loc[cons_id, "pq_z_mean_4500"] = (cons_info_test.loc[cons_id, "pq_z_mean"] > 4500).astype(float)
    cons_info_test.loc[cons_id, "variation_month_0085"] = (cons_info_test.loc[cons_id, "variation_month"] < 0.085).astype(float)

    
cons_info_test

100%|████████████████████████████████| 15379/15379 [02:22<00:00, 108.16it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,...,pq_fp_distance,pq_fg_cosine,pq_gp_cosine,pq_fp_cosine,pq_fz_cosine,pq_gz_cosine,pq_pz_cosine,variation_month,pq_z_mean_4500,variation_month_0085
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,1,0,...,2373.503318,0.076989,0.979375,0.957193,0.013026,0.027111,0.964811,0.980133,0.0,0.0
179406030,12.0,12.0,0.0,24,8431,8431,0,0,1,0,...,1860.847388,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.732051,0.0,0.0
179406094,2.0,2.0,1.0,24,8310,8310,0,0,1,0,...,2552.960243,0.062809,1.114785,1.105565,0.000572,0.051562,1.105591,1.496756,0.0,0.0
179406097,4.0,4.0,1.0,36,8341,8341,0,0,1,0,...,2116.962919,0.035125,1.198773,1.104793,0.005005,0.013703,1.140851,1.014585,0.0,0.0
179406099,2.0,2.0,1.0,24,8431,8431,0,0,1,0,...,1311.739685,0.017548,1.080833,1.068587,0.002274,0.007213,1.072008,0.969216,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,234.194364,0.179678,0.845413,0.918948,0.117407,0.007121,0.853454,1.253312,0.0,0.0
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.0,0.0
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,4487.578857,0.007583,0.009663,0.002064,0.000712,0.004659,0.001275,0.246757,1.0,0.0


<h3>日电量处理</h3>

In [109]:
cons_info_test["kwh_mean"] = 0
cons_info_test["kwh_median"] = 0
cons_info_test["kwh_std"] = 0
cons_info_test["kwh_std/mean"] = 0
cons_info_test["kwh_entropy"] = 0

cons_info_test["kwh_pap_r23_distance"] = 0
cons_info_test["kwh_pap_r24_distance"] = 0
cons_info_test["kwh_pap_r34_distance"] = 0

cons_info_test["kwh_pap_r23_cosine"] = 0
cons_info_test["kwh_pap_r24_cosine"] = 0
cons_info_test["kwh_pap_r34_cosine"] = 0

cons_info_test["kwh_pap_r1_cosine"] = 0
cons_info_test["kwh_pap_r2_cosine"] = 0
cons_info_test["kwh_pap_r3_cosine"] = 0
cons_info_test["kwh_pap_r4_cosine"] = 0
cons_info_test["variation_day"] = 0

In [110]:
elec_day_test = pd.read_csv("../data/测试组_比特币挖矿_日用电明细（20211217）.csv")
elec_day_test

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,179404030,2020-01-22 00:00:00,51.77,NaN,0.0,33.47,0.0,18.31
1,179404030,2020-01-23 00:00:00,73.27,NaN,0.0,51.83,0.0,21.44
2,179404030,2020-01-24 00:00:00,74.90,NaN,0.0,52.20,0.0,22.70
3,179404030,2020-01-25 00:00:00,62.73,NaN,0.0,37.27,0.0,25.45
4,179404030,2020-01-26 00:00:00,64.27,NaN,0.0,41.72,0.0,22.56
...,...,...,...,...,...,...,...,...
1660582,2880712108,2021-10-05 00:00:00,44.85,NaN,0.0,22.14,0.0,22.71
1660583,2880712108,2021-10-06 00:00:00,28.89,NaN,0.0,14.94,0.0,13.95
1660584,2880712108,2021-10-07 00:00:00,51.10,NaN,0.0,27.27,0.0,23.84
1660585,2880712108,2021-10-08 00:00:00,17.96,NaN,0.0,7.19,0.0,10.76


In [111]:
elec_day_test["rq"] = pd.to_datetime(elec_day_test["rq"])
elec_day_test

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,179404030,2020-01-22,51.77,NaN,0.0,33.47,0.0,18.31
1,179404030,2020-01-23,73.27,NaN,0.0,51.83,0.0,21.44
2,179404030,2020-01-24,74.90,NaN,0.0,52.20,0.0,22.70
3,179404030,2020-01-25,62.73,NaN,0.0,37.27,0.0,25.45
4,179404030,2020-01-26,64.27,NaN,0.0,41.72,0.0,22.56
...,...,...,...,...,...,...,...,...
1660582,2880712108,2021-10-05,44.85,NaN,0.0,22.14,0.0,22.71
1660583,2880712108,2021-10-06,28.89,NaN,0.0,14.94,0.0,13.95
1660584,2880712108,2021-10-07,51.10,NaN,0.0,27.27,0.0,23.84
1660585,2880712108,2021-10-08,17.96,NaN,0.0,7.19,0.0,10.76


In [112]:
for cons_id in tqdm(cons_info_test.index):
    df = elec_day_test[elec_day_test["id"] == cons_id]
    if df.shape[0] == 0:
        continue
    df = df.drop(columns=["id"]).sort_values("rq").set_index("rq")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    cons_info_test.loc[cons_id, "kwh_mean"] = df["kwh"].mean()
    cons_info_test.loc[cons_id, "kwh_median"] = df["kwh"].median()
    cons_info_test.loc[cons_id, "kwh_std"] = df["kwh"].std()
    cons_info_test.loc[(cons_info_test.index == cons_id) & (cons_info_test["kwh_mean"] > 0), "kwh_std/mean"] = cons_info_test["kwh_std"] / cons_info_test["kwh_mean"]
    cons_info_test.loc[cons_id, "kwh_entropy"] = entropy(df["kwh"])

    cons_info_test.loc[cons_id, "kwh_pap_r23_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info_test.loc[cons_id, "kwh_pap_r24_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info_test.loc[cons_id, "kwh_pap_r34_distance"] = distance.euclidean(df["kwh_pap_r3"], df["kwh_pap_r4"])

    cons_info_test.loc[cons_id, "kwh_pap_r23_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info_test.loc[cons_id, "kwh_pap_r24_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info_test.loc[cons_id, "kwh_pap_r34_cosine"] = cosine(df["kwh_pap_r3"], df["kwh_pap_r4"])
    
    cons_info_test.loc[cons_id, "kwh_pap_r1_cosine"] = cosine(df["kwh_pap_r1"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r2_cosine"] = cosine(df["kwh_pap_r2"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r3_cosine"] = cosine(df["kwh_pap_r3"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r4_cosine"] = cosine(df["kwh_pap_r4"], df["kwh"])
    
    cons_info.loc[cons_id, "variation_day"] = variation(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
cons_info_test

100%|█████████████████████████████████| 15379/15379 [04:08<00:00, 61.89it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,...,kwh_pap_r24_distance,kwh_pap_r34_distance,kwh_pap_r23_cosine,kwh_pap_r24_cosine,kwh_pap_r34_cosine,kwh_pap_r1_cosine,kwh_pap_r2_cosine,kwh_pap_r3_cosine,kwh_pap_r4_cosine,variation_day
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,1,0,...,87.325218,102.874561,1.000000,0.103449,1.000000,1.0,0.015134,1.000000,0.040248,0
179406030,12.0,12.0,0.0,24,8431,8431,0,0,1,0,...,0.000000,115.441143,1.000000,0.000000,1.000000,1.0,1.000000,0.000000,1.000000,0
179406094,2.0,2.0,1.0,24,8310,8310,0,0,1,0,...,161.682117,18.810803,1.000000,0.097529,1.000000,1.0,0.000859,1.000000,0.080452,0
179406097,4.0,4.0,1.0,36,8341,8341,0,0,1,0,...,77.606073,89.650691,1.000000,0.113770,1.000000,1.0,0.016689,1.000000,0.044281,0
179406099,2.0,2.0,1.0,24,8431,8431,0,0,1,0,...,52.523600,48.952625,1.000000,0.082221,1.000000,1.0,0.009680,1.000000,0.035987,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,106.542598,112.430454,1.000000,0.672559,1.000000,1.0,0.402164,1.000000,0.064085,0
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,1283.043257,1075.235788,0.002637,0.008105,0.009052,1.0,0.000969,0.001243,0.004624,0


In [113]:
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15379 entries, 179406029 to 2852368013
Data columns (total 53 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                15379 non-null  float64
 1   RUN_CAP                     15379 non-null  float64
 2   SHIFT_NO                    15379 non-null  float64
 3   CHK_CYCLE                   15379 non-null  int64  
 4   live_days                   15379 non-null  int64  
 5   check_days                  15379 non-null  int64  
 6   ELEC_TYPE_NAME_乡村居民生活用电     15379 non-null  uint8  
 7   ELEC_TYPE_NAME_商业用电         15379 non-null  uint8  
 8   ELEC_TYPE_NAME_城镇居民生活用电     15379 non-null  uint8  
 9   ELEC_TYPE_NAME_学校教学和学生生活用电  15379 non-null  uint8  
 10  ELEC_TYPE_NAME_居民生活用电       15379 non-null  uint8  
 11  ELEC_TYPE_NAME_普通工业         15379 non-null  uint8  
 12  ELEC_TYPE_NAME_非居民照明        15379 non-null  uint8  
 13  ELEC_TYPE_NAME_非工业

<h2>重新排列测试集</h2>

In [114]:
cons_info_test = cons_info_test.reindex(columns=cons_info.columns).drop(columns=["IS_FLAG"])
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15379 entries, 179406029 to 2852368013
Data columns (total 53 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                15379 non-null  float64
 1   RUN_CAP                     15379 non-null  float64
 2   SHIFT_NO                    15379 non-null  float64
 3   CHK_CYCLE                   15379 non-null  int64  
 4   live_days                   15379 non-null  int64  
 5   check_days                  15379 non-null  int64  
 6   ELEC_TYPE_NAME_乡村居民生活用电     15379 non-null  uint8  
 7   ELEC_TYPE_NAME_商业用电         15379 non-null  uint8  
 8   ELEC_TYPE_NAME_城镇居民生活用电     15379 non-null  uint8  
 9   ELEC_TYPE_NAME_学校教学和学生生活用电  15379 non-null  uint8  
 10  ELEC_TYPE_NAME_居民生活用电       15379 non-null  uint8  
 11  ELEC_TYPE_NAME_普通工业         15379 non-null  uint8  
 12  ELEC_TYPE_NAME_非居民照明        15379 non-null  uint8  
 13  ELEC_TYPE_NAME_非工业

In [115]:
x_test = ss.transform(cons_info_test.values)
x_test.shape

(15379, 53)

In [116]:
x_test

array([[-0.34570468, -0.35332375,  1.05812375, ...,  0.53828596,
        -0.20511538, -3.05641614],
       [-0.18021871, -0.18256316, -0.86811471, ..., -1.93452962,
         5.04390581, -3.05641614],
       [-0.34570468, -0.35332375,  1.05812375, ...,  0.53828596,
         0.0147641 , -3.05641614],
       ...,
       [11.53618784, 11.90728646, -0.86811471, ..., -1.93145671,
        -0.39995146, -3.05641614],
       [-0.18021871, -0.18256316, -0.86811471, ...,  0.53828596,
         0.30609063, -3.05641614],
       [ 0.48999946,  0.50901722, -0.86811471, ..., -1.93289553,
        -0.39812478, -3.05641614]])

<h2>预测数据</h2>

In [117]:
y_test = model.predict(x_test)
cons_info_test["label"] = y_test
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,...,kwh_pap_r34_distance,kwh_pap_r23_cosine,kwh_pap_r24_cosine,kwh_pap_r34_cosine,kwh_pap_r1_cosine,kwh_pap_r2_cosine,kwh_pap_r3_cosine,kwh_pap_r4_cosine,variation_day,label
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,0,0,1,0,...,102.874561,1.000000,0.103449,1.000000,1.0,0.015134,1.000000,0.040248,0,0
179406030,12.0,12.0,0.0,24,8431,8431,0,0,1,0,...,115.441143,1.000000,0.000000,1.000000,1.0,1.000000,0.000000,1.000000,0,0
179406094,2.0,2.0,1.0,24,8310,8310,0,0,1,0,...,18.810803,1.000000,0.097529,1.000000,1.0,0.000859,1.000000,0.080452,0,0
179406097,4.0,4.0,1.0,36,8341,8341,0,0,1,0,...,89.650691,1.000000,0.113770,1.000000,1.0,0.016689,1.000000,0.044281,0,0
179406099,2.0,2.0,1.0,24,8431,8431,0,0,1,0,...,48.952625,1.000000,0.082221,1.000000,1.0,0.009680,1.000000,0.035987,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,1,0,0,0,...,112.430454,1.000000,0.672559,1.000000,1.0,0.402164,1.000000,0.064085,0,0
2850017469,120.0,120.0,0.0,120,773,773,1,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,0
2850017472,720.0,720.0,0.0,120,773,773,1,0,0,0,...,1075.235788,0.002637,0.008105,0.009052,1.0,0.000969,0.001243,0.004624,0,0


In [118]:
cons_info_test.index.name = "id"
cons_info_test[["label"]].to_csv("../data/result20220210-1.csv", index=True)